In [1]:
from datasets import load_dataset
import polars as pl
import os
import cynde.functional as cf

In [2]:
# Get the directory above the current directory
above_dir = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(above_dir, "cache")
print(cache_dir)

c:\Users\Tommaso\Documents\Dev\Cynde\cache


In [3]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [4]:
df = pl.read_parquet(dataset_path+"_filtered.parquet")
df.head(1)


conversations,source
list[struct[3]],str
"[{""human"",""Given the following list of words, categorize them into 5 categories by similarity: ['Heart', 'Liver', 'Kidney', 'Brain', 'Stomach', 'Skin', 'Lung', 'Muscle', 'Intestine', 'Bone', 'Pancreas', 'Blood', 'Eye', 'Ear', 'Nose', 'Mouth', 'Esophagus', 'Throat', 'Spinal Cord', 'Bladder']"",null}, {""gpt"",""Here are the given words categorized into 5 groups based on their similarity: 1. Vital Organs: - Heart - Liver - Kidney - Lung - Pancreas 2. Digestive System Components: - Stomach - Intestine - Esophagus - Throat - Mouth 3. Sensory Organs: - Eye - Ear - Nose 4. Support and Movement: - Bone - Muscle - Spinal Cord - Skin 5. Body Fluids and Waste Management: - Blood - Bladder"",null}, … {""gpt"",""1) Vital Organs: - Diseases or disorders affecting one vital organ can impact the functioning of others. For example, heart failure can lead to fluid buildup in the lungs, affecting their function. Liver disease can cause toxins to accumulate in the bloodstream, potentially affecting brain function. Kidney disease can lead to electrolyte imbalances, which can impact heart function. - Organ transplantation involves replacing a damaged organ with a healthy one from a donor. Examples include heart, liver, kidney, lung, and pancreas transplants. Challenges include finding compatible donors, managing immune system rejection, and addressing surgical complications. 2) Digestive System Components: - Common digestive disorders include gastroesophageal reflux disease (GERD), irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), and peptic ulcers. Causes may include genetic factors, lifestyle choices, stress, infections, or immune system dysfunction. - The gut microbiome comprises trillions of microorganisms that aid in digestion, nutrient absorption, and immune system function. It can also influence mental health and contribute to the prevention or development of various diseases. 3) Sensory Organs: - Optical illusions work by exploiting the brain's visual processing system, causing it to misinterpret sensory information. They reveal that our perception of reality can be influenced by factors such as context, contrast, and patterns. - Animals' sensory organs can differ significantly from humans', offering unique capabilities. For example, dogs have a heightened sense of smell, eagles have exceptional long-distance vision, and bats use echolocation for navigation and foraging. 4) Support and Movement: - Exercise and physical activity promote bone and muscle health by stimulating bone growth, increasing bone density, and strengthening muscles. This helps prevent age-related bone loss and reduces the risk of injury. - Spinal cord injuries can have varying effects depending on the location and severity of the injury. Injuries can result in temporary or permanent loss of sensation, mobility, and other bodily functions. Paraplegia (paralysis of the legs and lower body) and quadriplegia (paralysis of all four limbs) are examples of potential outcomes of spinal cord injuries. 5) Body Fluids and Waste Management: - Blood types affect transfusions and organ transplants because of the presence of specific antigens on red blood cells. Incompatible blood types can cause an immune response, resulting in transfusion reactions or organ rejection. - Common bladder disorders include urinary incontinence, urinary tract infections (UTIs), interstitial cystitis, and bladder cancer. Treatments and management strategies can vary, including medication, lifestyle changes, physical therapy, or surgical interventions."",null}]","""caseus_custom"""


In [5]:
df["source"].value_counts()

source,count
str,u32
"""caseus_custom""",2688
"""LMSys Chatbot …",3136


In [6]:
df = df.with_columns(pl.col("conversations").list.eval(pl.element().struct.json_encode()).list.join("\n"),(pl.col("source")=="caseus_custom").alias("target"))

In [7]:
#check if the embedded df already exist at dataset_path+"_embedded.parquet"
if not os.path.exists(dataset_path+"_embedded_small_large.parquet"):
    embedded_df = cf.embed_columns(df, ["conversations"],models=["text-embedding-3-small","text-embedding-3-large"])
    embedded_df.write_parquet(dataset_path+"_embedded_small_large.parquet")
else:
    embedded_df = pl.read_parquet(dataset_path+"_embedded_small_large.parquet")

In [8]:
df["target"].value_counts()

target,count
bool,u32
true,2688
false,3136


In [9]:
cf.vanilla_kfold(embedded_df,group=None,k=5)

cv_index,fold_0,fold_1,fold_2,fold_3,fold_4
u32,str,str,str,str,str
943,"""test""","""train""","""train""","""train""","""train"""
2729,"""test""","""train""","""train""","""train""","""train"""
2223,"""test""","""train""","""train""","""train""","""train"""
4291,"""test""","""train""","""train""","""train""","""train"""
1022,"""test""","""train""","""train""","""train""","""train"""
3305,"""test""","""train""","""train""","""train""","""train"""
4465,"""test""","""train""","""train""","""train""","""train"""
3784,"""test""","""train""","""train""","""train""","""train"""
2899,"""test""","""train""","""train""","""train""","""train"""


df:pl.DataFrame, cv_type: Tuple[str,str], group_outer:List[str],k_outer:int,group_inner:List[str],k_inner:int,r_outer:int =1, r_inner:int =1):

In [10]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5},{"n_estimators": 50, "max_depth": 10}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embeddings"]},
         {"numerical":["conversations_text-embedding-3-large_embeddings"]},
         {"numerical":["conversations_text-embedding-3-small_embeddings","conversations_text-embedding-3-large_embeddings"]}]

In [11]:
results,pred=cf.train_nested_cv(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=1,
                     r_inner=1,
                     save_name="test")


cv_type: stratified
cv_type: stratified
['target', 'cv_index', 'outer_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_1']
Accuracy Test: 0.8729395604395604
Accuracy Val: 0.8633241758241759
MCC Test: 0.7441075503514313
MCC Val: 0.7247126421509431
Total CLS time: 00:00:00
Accuracy Test: 0.9103708791208791
Accuracy Val: 0.9189560439560439
MCC Test: 0.8195632322386872
MCC Val: 0.8369930128626445
Total CLS time: 00:00:01
Accuracy Test: 0.8863324175824175
Accuracy Val: 0.8853021978021978
MCC Test: 0.7718324552445832
MCC Val: 0.7696335692960288
Total CLS time: 00:00:00
Accuracy Test: 0.9395604395604396
Accuracy Val: 0.947115384615

In [12]:
summary = cf.results_summary(results,by_test_fold=True)
summary

classifier,classifier_hp,input_features_name,r_outer,r_inner,mcc_train,mcc_val,mcc_test,accuracy_train,accuracy_val,accuracy_test,n
str,str,str,i64,i64,f64,f64,f64,f64,f64,f64,u32
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,0.999655,0.888123,0.867271,0.999828,0.944368,0.93398,4
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,1.0,0.887443,0.878014,1.0,0.944025,0.939303,4
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,0.998965,0.839564,0.83441,0.999485,0.920158,0.917668,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.929292,0.776767,0.776048,0.964801,0.888908,0.88865,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.925072,0.76877,0.774371,0.96274,0.884959,0.887792,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.903944,0.740154,0.730889,0.952266,0.870879,0.866243,4


In [13]:
for res in summary.sort(by="mcc_val",descending=True).rows(named=True):
    print(res["input_features_name"],res["classifier_hp"],res["mcc_val"])

conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings n_estimators_50_max_depth_10 0.8881234780421673
conversations_text-embedding-3-large_embeddings n_estimators_50_max_depth_10 0.8874427211049117
conversations_text-embedding-3-small_embeddings n_estimators_50_max_depth_10 0.8395639695212582
conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings n_estimators_10_max_depth_5 0.7767669477836909
conversations_text-embedding-3-large_embeddings n_estimators_10_max_depth_5 0.7687696735765799
conversations_text-embedding-3-small_embeddings n_estimators_10_max_depth_5 0.7401544689416826
